Import all required libraries

In [3]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
bucket_name = 'testbankapplication' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-1


In [5]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [6]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://testbankapplication/xgboost-as-a-built-in-algo/output


In [7]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [9]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [12]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [13]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [20]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
import boto3
import sagemaker

region = boto3.Session().region_name

# Using the sagemaker.image_uris module to retrieve the image URI
container = sagemaker.image_uris.retrieve(
    framework='xgboost',
    region=region,
    version='1.0-1'
)

print(container)

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3


In [21]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [24]:
# Updated Estimator with new parameter names
estimator = sagemaker.estimator.Estimator(
    image_uri=container,
    hyperparameters=hyperparameters,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # 5 GB
    output_path=output_path,
    use_spot_instances=True,
    max_run=300,
    max_wait=600
)

In [25]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-07-15-05-59-45-941


2024-07-15 05:59:46 Starting - Starting the training job...
2024-07-15 06:00:02 Starting - Preparing the instances for training...
2024-07-15 06:00:29 Downloading - Downloading input data...
2024-07-15 06:00:54 Downloading - Downloading the training image...
2024-07-15 06:01:45 Training - Training image download completed. Training in progress.
2024-07-15 06:01:45 Uploading - Uploading generated training model[2024-07-15 06:01:36.940 ip-10-0-220-161.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root

In [27]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-07-15-06-08-51-620
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-07-15-06-08-51-620
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-07-15-06-08-51-620


------!

In [28]:
print(xgb_predictor)

Predictor: {'endpoint_name': 'sagemaker-xgboost-2024-07-15-06-08-51-620', 'sagemaker_session': <sagemaker.session.Session object at 0x7f57126b1e10>, 'serializer': <sagemaker.base_serializers.IdentitySerializer object at 0x7f5715797a60>, 'deserializer': <sagemaker.base_deserializers.BytesDeserializer object at 0x7f57157b41f0>}


In [30]:
import numpy as np
import pandas as pd
from sagemaker.serializers import CSVSerializer
from sagemaker.predictor import Predictor

# Assuming test_data is a pandas DataFrame
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values  # load the data into an array

# Assuming xgb_predictor is already defined and is an instance of sagemaker.predictor.Predictor
xgb_predictor.content_type = 'text/csv'  # set the data type for an inference
xgb_predictor.serializer = CSVSerializer()  # set the serializer type

# Making predictions
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')  # predict!
predictions_array = np.fromstring(predictions, sep=',')  # and turn the prediction into an array

print(predictions_array.shape)

(12357,)


In [31]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [32]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [33]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-07-15-06-08-51-620


[{'ResponseMetadata': {'RequestId': '3SRYANVQY24CMDH1',
   'HostId': '8DZjxmJM3w3Dn0+W7iyDH6VVfnzEoLmC1onFMjehrTLjjiPl623SpiSFVa1J6WI5tC6gFaJLDCD2TKVIlZBCNcwymWX9xCJn/qwd96SYRrE=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '8DZjxmJM3w3Dn0+W7iyDH6VVfnzEoLmC1onFMjehrTLjjiPl623SpiSFVa1J6WI5tC6gFaJLDCD2TKVIlZBCNcwymWX9xCJn/qwd96SYRrE=',
    'x-amz-request-id': '3SRYANVQY24CMDH1',
    'date': 'Mon, 15 Jul 2024 06:26:53 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-07-15-05-59-45-941/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-07-15-05-59-45-941/debug-output/claim.smd'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2024-07-15-05-59-45-941/profiler-output/framework/training_job_end.ts'},
   {'Key':